In [7]:
import sys
from pathlib import Path
import numpy as np
import pandas as pd

# Project root (notebooks/ is one level down)
PROJECT_ROOT = Path("..").resolve()


In [8]:
# Load macro-conditioned PH data from Notebook 06
df = pd.read_csv(
    PROJECT_ROOT / "powerbi/exports/PH_XAU_8p88y_MacroConditioned.csv",
    parse_dates=["date"]
)

df = df.set_index("date").sort_index()

df.head()


,cycle_strength,window,m,tau,representation,cyclic_regime,CPI,M2,FedFunds,inflation_yoy,inflation_accel,inflation_regime,m2_yoy,liquidity_regime,rate_regime
date,,,,,,,,,,,,,,,
1819-12-31,0.000000,107,12,3,log_price,Non-Cyclic,NaN,NaN,NaN,NaN,NaN,Decelerating,NaN,Tight,Low
1820-03-31,0.000000,107,12,3,log_price,Non-Cyclic,NaN,NaN,NaN,NaN,NaN,Decelerating,NaN,Tight,Low
1820-03-31,0.012543,107,12,3,log_returns,Non-Cyclic,NaN,NaN,NaN,NaN,NaN,Decelerating,NaN,Tight,Low
1820-06-30,0.012543,107,12,3,log_returns,Non-Cyclic,NaN,NaN,NaN,NaN,NaN,Decelerating,NaN,Tight,Low
1820-06-30,0.000000,107,12,3,log_price,Non-Cyclic,NaN,NaN,NaN,NaN,NaN,Decelerating,NaN,Tight,Low


In [9]:
required = [
    "cycle_strength",
    "cyclic_regime",
    "inflation_regime",
    "liquidity_regime",
    "rate_regime",
]

missing = [c for c in required if c not in df.columns]
missing


[]

In [10]:
def summarize_group(df, group_col, y_col="cycle_strength"):
    g = df.groupby(group_col)[y_col]
    out = pd.DataFrame({
        "n": g.size(),
        "mean": g.mean(),
        "median": g.median(),
        "std": g.std(),
    })
    out["se_mean"] = out["std"] / np.sqrt(out["n"])
    return out.sort_index()

summ_infl = summarize_group(df, "inflation_regime")
summ_liq  = summarize_group(df, "liquidity_regime")
summ_rate = summarize_group(df, "rate_regime")

summ_infl, summ_liq, summ_rate


(                     n      mean    median       std   se_mean
 inflation_regime                                              
 Accelerating       344  0.025987  0.011388  0.029453  0.001588
 Decelerating      2231  0.020010  0.007645  0.031936  0.000676,
                      n      mean    median       std   se_mean
 liquidity_regime                                              
 Loose              724  0.026345  0.010842  0.029610  0.001100
 Tight             1851  0.018643  0.007272  0.032198  0.000748,
                 n      mean    median       std   se_mean
 rate_regime                                              
 High          746  0.026402  0.011499  0.028458  0.001042
 Low          1829  0.018527  0.007076  0.032631  0.000763)

In [11]:
def perm_test_diff_means(x, y, n_perm=20000, seed=7):
    rng = np.random.default_rng(seed)
    x = np.asarray(x, dtype=float)
    y = np.asarray(y, dtype=float)
    x = x[~np.isnan(x)]
    y = y[~np.isnan(y)]

    obs = x.mean() - y.mean()

    pooled = np.concatenate([x, y])
    n_x = len(x)

    diffs = np.empty(n_perm, dtype=float)
    for i in range(n_perm):
        rng.shuffle(pooled)
        diffs[i] = pooled[:n_x].mean() - pooled[n_x:].mean()

    p = (np.sum(np.abs(diffs) >= abs(obs)) + 1) / (n_perm + 1)
    return obs, p


In [12]:
def bootstrap_diff_means(x, y, n_boot=10000, seed=7):
    rng = np.random.default_rng(seed)
    x = np.asarray(x, dtype=float)
    y = np.asarray(y, dtype=float)
    x = x[~np.isnan(x)]
    y = y[~np.isnan(y)]

    diffs = np.empty(n_boot, dtype=float)
    for i in range(n_boot):
        xb = rng.choice(x, size=len(x), replace=True)
        yb = rng.choice(y, size=len(y), replace=True)
        diffs[i] = xb.mean() - yb.mean()

    lo, hi = np.quantile(diffs, [0.025, 0.975])
    return diffs.mean(), (lo, hi)


In [13]:
x = df.loc[df["liquidity_regime"] == "Loose", "cycle_strength"]
y = df.loc[df["liquidity_regime"] == "Tight", "cycle_strength"]

obs, p_perm = perm_test_diff_means(x, y, n_perm=20000, seed=1)
boot_mean, (ci_lo, ci_hi) = bootstrap_diff_means(x, y, n_boot=10000, seed=1)

print("Liquidity (Loose - Tight)")
print("Observed mean diff:", obs)
print("Permutation p-value:", p_perm)
print("Bootstrap mean diff:", boot_mean)
print("95% bootstrap CI:", (ci_lo, ci_hi))


Liquidity (Loose - Tight)
Observed mean diff: 0.007702393305123397
Permutation p-value: 4.999750012499375e-05
Bootstrap mean diff: 0.0077002610468260785
95% bootstrap CI: (np.float64(0.005101835972718333), np.float64(0.01030943775587992))


In [14]:
x = df.loc[df["rate_regime"] == "High", "cycle_strength"]
y = df.loc[df["rate_regime"] == "Low", "cycle_strength"]

obs, p_perm = perm_test_diff_means(x, y, n_perm=20000, seed=2)
boot_mean, (ci_lo, ci_hi) = bootstrap_diff_means(x, y, n_boot=10000, seed=2)

print("Rates (High - Low)")
print("Observed mean diff:", obs)
print("Permutation p-value:", p_perm)
print("Bootstrap mean diff:", boot_mean)
print("95% bootstrap CI:", (ci_lo, ci_hi))


Rates (High - Low)
Observed mean diff: 0.007875617400621197
Permutation p-value: 4.999750012499375e-05
Bootstrap mean diff: 0.00786874783043367
95% bootstrap CI: (np.float64(0.005292427636098228), np.float64(0.010368978391090787))


In [15]:
x = df.loc[df["inflation_regime"] == "Accelerating", "cycle_strength"]
y = df.loc[df["inflation_regime"] == "Decelerating", "cycle_strength"]

obs, p_perm = perm_test_diff_means(x, y, n_perm=20000, seed=3)
boot_mean, (ci_lo, ci_hi) = bootstrap_diff_means(x, y, n_boot=10000, seed=3)

print("Inflation (Accelerating - Decelerating)")
print("Observed mean diff:", obs)
print("Permutation p-value:", p_perm)
print("Bootstrap mean diff:", boot_mean)
print("95% bootstrap CI:", (ci_lo, ci_hi))


Inflation (Accelerating - Decelerating)
Observed mean diff: 0.005977088963284197
Permutation p-value: 0.001399930003499825
Bootstrap mean diff: 0.005958933068549116
95% bootstrap CI: (np.float64(0.0025649043297335163), np.float64(0.009355850794634888))


In [16]:
def test_and_store(group_col, a, b, seed):
    x = df.loc[df[group_col] == a, "cycle_strength"]
    y = df.loc[df[group_col] == b, "cycle_strength"]
    obs, p = perm_test_diff_means(x, y, n_perm=20000, seed=seed)
    return obs, p

obs_liq, p_liq = test_and_store("liquidity_regime", "Loose", "Tight", seed=10)
obs_rate, p_rate = test_and_store("rate_regime", "High", "Low", seed=11)
obs_infl, p_infl = test_and_store("inflation_regime", "Accelerating", "Decelerating", seed=12)

results = pd.DataFrame({
    "Test": ["Liquidity (Loose–Tight)", "Rates (High–Low)", "Inflation (Accel–Decel)"],
    "Observed Mean Diff": [obs_liq, obs_rate, obs_infl],
    "Permutation p": [p_liq, p_rate, p_infl],
})

results["Bonferroni p"] = np.minimum(results["Permutation p"] * 3, 1.0)
results


,Test,Observed Mean Diff,Permutation p,Bonferroni p
0,Liquidity (Loose–Tight),0.007702,0.00005,0.00015
1,Rates (High–Low),0.007876,0.00005,0.00015
2,Inflation (Accel–Decel),0.005977,0.00100,0.00300
